In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, GaussianDropout, Flatten, SimpleRNN, LSTM
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, PReLU, ELU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import Activation
import math

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv')
df.head(10)

def getTrainTest(url, splitPercent=0.8):
    df = pd.read_csv(url, usecols=[1], engine='python')
    print(df)
    data = np.array(df.values.astype('float32'))
    print(data)
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n * splitPercent)
    trainData = data[range(split)]
    testData = data[split:]
    return trainData, testData, data

In [21]:
sunspotsData = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
trainData, testData, data = getTrainTest(sunspotsData)

def createNN(input_shape, activation):
    model = Sequential()
    
    model.add(Dense(
        64,
        input_shape=input_shape,
        activation=activation[0]
    ))

    model.add(Flatten())
    
    model.add(Dense(
        32,
        input_shape=input_shape,
        activation=activation[0]
    ))
    
    model.add(Flatten())
    
    model.add(Dense(
        1,
        input_shape=input_shape,
        activation=activation[0]
    ))
   
    model.add(Flatten())
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def createRNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()

    model.add(SimpleRNN(
        hidden_units, 
        input_shape=input_shape, 
        activation=activation[0]
    ))

    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def createLSTM(hidden_units, dense_units, input_shape, activation):
    model = Sequential()

    model.add(LSTM(
        hidden_units, 
        input_shape=input_shape, 
        activation=activation[0]
    ))

    model.add(Dense(units=dense_units, activation=activation[1]))

    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

      Sunspots
0         58.0
1         62.6
2         70.0
3         55.7
4         85.0
...        ...
2815      71.8
2816      50.3
2817      55.8
2818      33.3
2819      33.4

[2820 rows x 1 columns]
[[58. ]
 [62.6]
 [70. ]
 ...
 [55.8]
 [33.3]
 [33.4]]


In [22]:
def getXY(dat, timeSteps):
    # Indices of target array
    indY = np.arange(timeSteps, len(dat), timeSteps)
    Y = dat[indY]

    # Prepare X
    rowsX = len(Y)
    X = dat[range(timeSteps * rowsX)]
    X = np.reshape(X, (rowsX, timeSteps, 1))   

    return X, Y

def printError(trainY, testY, trainPredict, testPredict):    
    # Error of predictions
    trainRMSE = math.sqrt(mean_squared_error(trainY, trainPredict))
    trainRMAE = math.sqrt(mean_absolute_error(trainY, trainPredict))
    testRMSE = math.sqrt(mean_squared_error(testY, testPredict))
    testRMAE = math.sqrt(mean_absolute_error(testY, testPredict))
    testR2 = r2_score(testY, testPredict)

    print('Train RMSE: %.3f RMSE' % (trainRMSE))
    print('Train RMAE: %.3f RMSE' % (trainRMAE))
    print('Test RMSE: %.3f RMSE' % (testRMSE))   
    print('Test RMAE: %.3f RMSE' % (testRMAE))   
    print('Test R2: %.3f R2' % (testR2))
 
timeSteps = 12
trainX, trainY = getXY(trainData, timeSteps)
testX, testY = getXY(testData, timeSteps)

In [23]:
modelRNN = createRNN(hidden_units=3, dense_units=1, input_shape=(timeSteps,1), activation=['tanh', 'tanh'])
modelNN = createNN(input_shape=(timeSteps,1), activation=['relu', 'relu'])
modelLSTM = createLSTM(hidden_units=3, dense_units=1, input_shape=(timeSteps,1), activation=['tanh', 'tanh'])

modelRNN.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)
modelNN.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)
modelLSTM.fit(trainX, trainY, epochs=20, batch_size=1, verbose=2)

Epoch 1/20
187/187 - 9s - loss: 0.0273 - 9s/epoch - 48ms/step
Epoch 2/20
187/187 - 8s - loss: 0.0072 - 8s/epoch - 44ms/step
Epoch 3/20
187/187 - 8s - loss: 0.0053 - 8s/epoch - 44ms/step
Epoch 4/20
187/187 - 9s - loss: 0.0048 - 9s/epoch - 47ms/step
Epoch 5/20
187/187 - 9s - loss: 0.0045 - 9s/epoch - 47ms/step
Epoch 6/20
187/187 - 8s - loss: 0.0044 - 8s/epoch - 45ms/step
Epoch 7/20
187/187 - 8s - loss: 0.0043 - 8s/epoch - 44ms/step
Epoch 8/20
187/187 - 8s - loss: 0.0043 - 8s/epoch - 44ms/step
Epoch 9/20
187/187 - 8s - loss: 0.0042 - 8s/epoch - 45ms/step
Epoch 10/20
187/187 - 9s - loss: 0.0041 - 9s/epoch - 46ms/step
Epoch 11/20
187/187 - 8s - loss: 0.0041 - 8s/epoch - 45ms/step
Epoch 12/20
187/187 - 8s - loss: 0.0040 - 8s/epoch - 45ms/step
Epoch 13/20
187/187 - 8s - loss: 0.0040 - 8s/epoch - 44ms/step
Epoch 14/20
187/187 - 8s - loss: 0.0039 - 8s/epoch - 44ms/step
Epoch 15/20
187/187 - 8s - loss: 0.0038 - 8s/epoch - 43ms/step
Epoch 16/20
187/187 - 8s - loss: 0.0039 - 8s/epoch - 44ms/step
E

In [24]:
trainPredictNN = modelNN.predict(trainX)
testPredictNN = modelNN.predict(testX)

trainPredictRNN = modelRNN.predict(trainX)
testPredictRNN = modelRNN.predict(testX)

trainPredictLSTM = modelLSTM.predict(trainX)
testPredictLSTM = modelLSTM.predict(testX)

print('\nNN')
printError(trainY, testY, trainPredictNN, testPredictNN)

print('\nRNN')
printError(trainY, testY, trainPredictRNN, testPredictRNN)

print('\nLSTM')
printError(trainY, testY, trainPredictLSTM, testPredictLSTM)

2/2 [==============================] - 0s 13ms/step

NN
Train RMSE: 0.050 RMSE
Train RMAE: 0.196 RMSE
Test RMSE: 0.084 RMSE
Test RMAE: 0.251 RMSE


NameError: name 'test_r2' is not defined